In [258]:
## 라이브러리 import
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

### 데이터 로드

In [259]:
sample = pd.read_csv('sample_submission.csv')
test_df = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')

### 필요 없는 Columns 제거

In [260]:
## 'FLAG_MOBIL' 컬럼은 모두 1(휴대폰 모두 소유). 따라서 제거
test_df.drop(['FLAG_MOBIL'], axis=1, inplace=True)
df.drop(['FLAG_MOBIL'], axis=1, inplace=True)

## 'occyp_type' 컬럼 유형은 결측값 존재해서 일단 제거
test_df.drop(['occyp_type'], axis=1, inplace=True)
df.drop(['occyp_type'], axis=1, inplace=True)

## 여기 컬럼들은 간다하게 적용하기 그래서 제거
## 추후 여기 지워야
test_df.drop(['Education','family_type','gender','house_type','income_type'], axis=1, inplace=True)
df.drop(['Education','family_type','gender','house_type','income_type'], axis=1, inplace=True)

### 간단한 머신러닝 RF 적용

In [261]:
## train_test 셋 분리
features = list(df.columns.difference(['credit']))
target = df['credit']

X = df[features]
y = target.copy()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [262]:
#개수 확인
print(X_train.shape, X_test.shape,y_train.shape,y_test.shape)

(21165, 9) (5292, 9) (21165,) (5292,)


In [263]:
# ML 적용
rf = RandomForestClassifier(random_state = 0)

rf.fit(X_train, y_train)
prd = rf.predict(X_test)

In [264]:
# 결과
print(accuracy_score(y_test,prd))

0.7033257747543462


### 본 test셋 적용

In [265]:
## test셋 예측
prd = rf.predict(test_df)

In [266]:
## one-hot-encoding 적용
onehot = OneHotEncoder(sparse=False)
prd = prd.reshape(-1,1)
onehot.fit(prd)
onehot_encode = onehot.transform(prd)
test_y_df = pd.DataFrame(onehot_encode)

In [267]:
## 제출을 위해 정리
test_y_df.columns = [0,2]
test_y_df[1] = 0
test_y_df = test_y_df[[0,1,2]]
test_y_df.columns = [0,1,2]
sample[['0','1','2']] = test_y_df
sample

,index,0,1,2
0,26457,1.0,0,0.0
1,26458,1.0,0,0.0
2,26459,1.0,0,0.0
3,26460,1.0,0,0.0
4,26461,1.0,0,0.0
...,...,...,...,...
9995,36452,1.0,0,0.0
9996,36453,1.0,0,0.0
9997,36454,1.0,0,0.0
9998,36455,1.0,0,0.0


In [270]:
sample.to_csv('first.csv', index=False)